# EnergyAnalysis Class
Calculate the solar supply, demand, storage left, and storage demand with given data of area (default as California)
...

## Import packages

In [44]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

## Class implementation

### Energy Supply Analysis

In [26]:
class EnergySupplyAnalysis:
    '''
    The class for energy supply analysis for given capacity and GHI data

    Parameter
    ---------
    capacity: int
        Utility-scale solar power capacity
        unit: Mega Watts
    
    solarDataFile: str
        Solar data file path
    '''
    def __init__(self, capacity=17500, solarDataFile: str='../raw_data/midCalifornia_GHI.csv') -> None:
        assert isinstance(capacity, float) or isinstance(capacity, int)
        assert isinstance(solarDataFile, str)

        self.solarData = pd.read_csv(solarDataFile)
        self.capacity = capacity
        self.__energySupply()

    def __energySupply(self) -> pd.DataFrame:
        '''
        Calculate the supply with given solar and solar panel capacity data

        Returns
        -------
        pandas.DataFrame
            | Year | Month | Day | Hour | Minute | GHI | Supply |
            |------|-------|-----|------|--------|-----|--------|
        
            Year: int
            Month: int
            Day: int
            Hour: int
            Minute: int
            GHI: int
            Supply: float
        '''
        self.__supplyDF = self.solarData
        self.__supplyDF['Supply'] = self.__supplyDF['GHI'] * self.capacity / 1000
    
    def getEnergySupply(self) -> pd.DataFrame:
        return self.__supplyDF

### Energy Demand Analysis

In [69]:
class EnergyDemandAnalysis:
    '''
    The class for energy demand analysis for given electrical data

    Parameter
    ---------
    energyDataFile: str
        Electrical demand data file path
    '''
    def __init__(self, energyDataFile: str='../raw_data/CAISOactualLoad.csv') -> None:
        assert isinstance(energyDataFile, str)

        self.demandData = pd.read_csv(energyDataFile)
        self.__energyDemand()

    def __energyDemand(self) -> pd.DataFrame:
        '''
        Calculate the demand with electrical data

        Returns
        -------
        pandas.DataFrame
            | Year | Month | Day | Hour | Minute | Demand |
            |------|-------|-----|------|--------|--------|
        
            Year: int
            Month: int
            Day: int
            Hour: int
            Minute: int
            GHI: int
            Demand: float
        '''

        data = self.demandData[self.demandData['zone'] == 'CA ISO']
        data['date'] = data.apply(lambda row: datetime.strptime(row['Date'], '%m/%d/%Y %I:%M:%S %p'), axis=1)
        data['Year'] = data.apply(lambda row: row['date'].year, axis=1)
        data['Month'] = data.apply(lambda row: row['date'].month, axis=1)
        data['Day'] = data.apply(lambda row: row['date'].day, axis=1)
        data['Hour'] = data.apply(lambda row: row['date'].hour, axis=1)
        data['Minute'] = data.apply(lambda row: row['date'].minute, axis=1)
        self.__demandDF = data[['Year', 'Month', 'Day', 'Hour', 'Minute', 'load']]
        self.__demandDF = self.__demandDF.rename(columns={"load": "Demand"})
    
    def getEnergyDemand(self) -> pd.DataFrame:
        return self.__demandDF

In [ ]:
class EnergySupplyPlot:
    '''
    Plot energy supply analysis for given capacity and GHI data
    '''
    def __init__(self, energySupply: pd.DataFrame) -> None:
        assert isinstance(energySupply, pd.DataFrame)
        

        self.solarData = pd.read_csv(solarDataFile)
        self.energyData = pd.read_csv(energyDataFile)
        self.capacity = capacity
        self.storage = storage
        self.__energyDemand()

    def __energyDemand(self) -> pd.DataFrame:
        '''
        Calculate the supply, demand, storageLeft, storageDemand with given data

        Returns
        -------
        pandas.DataFrame
            | date | hour | supply | demand | storageLeft | storageDemand |
            |------|------|--------|--------|-------------|---------------|
        
            date: str
                %m/%d/%Y
                04/12/2018
            hour: int
            supply: float
            demand: float
            storageLeft: float
            storageDemand: float
        '''
        outputs = pd.DataFrame()
        
        self.__energyDF = outputs

    def plotAnalysis(self, startTime: datetime=None, endTime: datetime=None, unit: str='hour') -> None:
        '''
        Plot the supply, demand, storage left, and storage demand for given period and unit

        Parameter
        ---------
        startTime: str
            %m-%d-%Y %H:%M
            ex: 04/12/2018 00:00

        endTime: str
            %m-%d-%Y %H:%M
            ex: 04/12/2018 00:00

        unit: str
            The unit can be 'hour', 'day', or 'year'
        '''
        assert isinstance(startTime, str)
        assert isinstance(endTime, str)
        assert unit == 'hour' or unit == 'day' or unit == 'year'
 

    def getEnergeDF(self) -> pd.DataFrame:
        return self._energyDF
    
    def getEnergySupply(self) -> pd.DataFrame:
        return self.__energyDF['supply']
    
    def getEnergyDemand(self) -> pd.DataFrame:
        return self.__energyDF['demand']
    
    def getStorageLeft(self) -> pd.DataFrame:
        return self.__energyDF['storageLeft']
    
    def getStorageDemand(self) -> pd.DataFrame:
        return self.__energyDF['storageDemand']




In [29]:
test = EnergySupplyAnalysis()
test.getEnergySupply()

,Year,Month,Day,Hour,Minute,GHI,Supply
0,2018,1,1,0,30,25,437.5
1,2018,1,1,1,30,0,0.0
2,2018,1,1,2,30,0,0.0
3,2018,1,1,3,30,0,0.0
4,2018,1,1,4,30,0,0.0
...,...,...,...,...,...,...,...
26275,2020,12,31,19,30,509,8907.5
26276,2020,12,31,20,30,517,9047.5
26277,2020,12,31,21,30,464,8120.0
26278,2020,12,31,22,30,356,6230.0


In [ ]:
class EnergySupply:
    '''
    Write sth...
    '''
    def __init__(self, capacity, storage, solarDataFile: str='../raw_data/CAISOdata.csv', energyDataFile: str='../raw_data/CAISOactualLoad.csv') -> None:
        assert isinstance(capacity, float) or isinstance(capacity, int)
        assert isinstance(storage, float) or isinstance(storage, int)
        assert isinstance(solarDataFile, str)
        assert isinstance(energyDataFile, str)

        self.solarData = pd.read_csv(solarDataFile)
        self.energyData = pd.read_csv(energyDataFile)
        self.capacity = capacity
        self.storage = storage
        self.__energyDemand()

    def __energyDemand(self) -> pd.DataFrame:
        '''
        Calculate the supply, demand, storageLeft, storageDemand with given data

        Returns
        -------
        pandas.DataFrame
            | date | hour | supply | demand | storageLeft | storageDemand |
            |------|------|--------|--------|-------------|---------------|
        
            date: str
                %m/%d/%Y
                04/12/2018
            hour: int
            supply: float
            demand: float
            storageLeft: float
            storageDemand: float
        '''
        outputs = pd.DataFrame()
        
        self.__energyDF = outputs

    def plotAnalysis(self, startTime: datetime=None, endTime: datetime=None, unit: str='hour') -> None:
        '''
        Plot the supply, demand, storage left, and storage demand for given period and unit

        Parameter
        ---------
        startTime: str
            %m-%d-%Y %H:%M
            ex: 04/12/2018 00:00

        endTime: str
            %m-%d-%Y %H:%M
            ex: 04/12/2018 00:00

        unit: str
            The unit can be 'hour', 'day', or 'year'
        '''
        assert isinstance(startTime, str)
        assert isinstance(endTime, str)
        assert unit == 'hour' or unit == 'day' or unit == 'year'
 

    def getEnergeDF(self) -> pd.DataFrame:
        return self._energyDF
    
    def getEnergySupply(self) -> pd.DataFrame:
        return self.__energyDF['supply']
    
    def getEnergyDemand(self) -> pd.DataFrame:
        return self.__energyDF['demand']
    
    def getStorageLeft(self) -> pd.DataFrame:
        return self.__energyDF['storageLeft']
    
    def getStorageDemand(self) -> pd.DataFrame:
        return self.__energyDF['storageDemand']




In [ ]:
#Energy storage analysis:

class StorageAnalysis:
    """
        This class takes in a pandas dataframe in the form of 
            | date | hour | supply |
            |------|------|--------|
            and
            | date | hour | demand |
            |------|------|--------|

            and creates a dataframe in the form of:
            
            | date | hour | storage demand | storage supplied | storageLeft |
            |------|------|----------------|------------------|-------------|

            where storage demand is the need for power that is not met by solar capacity,
            storage supplied is the amount of energy supplied that hour, which should equal demand unless storageLeft goes to zero.
            storageLeft is a the amount of energy left in the storage devices.
            
    """
    def __init__(self, the_supply: pd.DataFrame, the_demand: pd.DataFrame, the_storageMWh: int):
        self.supply = the_supply
        self


## Test

In [23]:
test = EnergyAnalysis(10, 12)

""
